In [1]:
import itertools
from collections import defaultdict
import pandas as pd

In [7]:
print(3 ** 6)

729


In [8]:
# 기본 변수 설정
Cases = ['W', 'D', 'L']  # W=Win, D=Draw, L=Loss
Initial_Teams = {'인천': 36, '전북': 38, '광주': 45, '대전': 42, '대구': 40, '제주': 48}
Matches = [
    [['제주', '광주'], ['전북', '대구'], ['인천', '대전']],  # 37라운드
    [['대전', '제주'], ['대구', '인천'], ['광주', '전북']]   # 38라운드
]

In [9]:
# 결과 저장 리스트
results = []

# 점수 업데이트 함수
def update_points(teams, match, result):
    team1, team2 = match
    if result == 'W':  # team1 승리
        teams[team1] += 3
    elif result == 'D':  # 무승부
        teams[team1] += 1
        teams[team2] += 1
    elif result == 'L':  # team2 승리
        teams[team2] += 3

In [10]:
# 각 라운드별 경우의 수 시뮬레이션
current_round_results = [Initial_Teams]  # 첫 번째 라운드는 초기 점수로 시작
round_results = []

for round_num, round_matches in enumerate(Matches, start=37):
    next_round_results = []  # 다음 라운드로 전달할 결과 저장

    for team_scores in current_round_results:
        # 각 라운드 내 경기의 경우의 수 계산
        match_cases = list(itertools.product(Cases, repeat=len(round_matches)))
        
        for case in match_cases:
            temp_teams = team_scores.copy()  # 이전 라운드 결과를 복사하여 사용
            for match, result in zip(round_matches, case):
                update_points(temp_teams, match, result)
            
            # 38라운드의 최종 결과만 저장
            if round_num == 38:
                sorted_teams = sorted(temp_teams.items(), key=lambda x: x[1], reverse=True)
                # 각 경우의 수에 대해 순위와 점수를 데이터로 저장
                round_results.append({
                    "경우의 수": case,
                    **{team: points for team, points in sorted_teams}
                })

            next_round_results.append(temp_teams)  # 다음 라운드에 전달할 결과 저장

    current_round_results = next_round_results  # 다음 라운드로 점수 전달

In [11]:
# 결과 출력
for round_num, case, ranking in results:
    print(f"{round_num}라운드 경기 결과: {case}")
    for rank, (team, points) in enumerate(ranking, start=1):
        print(f"{rank}위: {team} - {points}점")
    print()

In [12]:
# DataFrame 생성
df_38_round = pd.DataFrame(round_results)

df_38_round = df_38_round.drop(['경우의 수'], axis = 1)

df_38_round

,제주,광주,대전,대구,전북,인천
0,51,48,45,43,41,39
1,51,46,45,43,42,39
2,51,45,45,43,44,39
3,51,48,45,41,41,40
4,51,46,45,41,42,40
...,...,...,...,...,...,...
724,51,49,45,44,39,37
725,51,48,45,44,41,37
726,51,51,45,43,38,39
727,51,49,45,43,39,39


In [ ]:
# 1) '인천' 이 단독 꼴찌할 경우의 수
df_1 = df_38_round[
    (
        (df_38_round['인천'] < df_38_round['전북']) &
        (df_38_round['인천'] < df_38_round['광주']) &
        (df_38_round['인천'] < df_38_round['대전']) &
        (df_38_round['인천'] < df_38_round['대구']) &
        (df_38_round['인천'] < df_38_round['제주'])
    )
]

# 결과 확인
df_1


,제주,광주,대전,대구,전북,인천
0,51,48,45,43,41,39
1,51,46,45,43,42,39
2,51,45,45,43,44,39
3,51,48,45,41,41,40
4,51,46,45,41,42,40
...,...,...,...,...,...,...
722,51,48,45,46,41,36
723,51,51,45,44,38,37
724,51,49,45,44,39,37
725,51,48,45,44,41,37


In [16]:
486 / 729 * 100

66.66666666666666

In [17]:
# 2) '전북' 점수가 가장 낮은 DataFrame
df_2 = df_38_round[
    (
        (df_38_round['전북'] < df_38_round['인천']) &
        (df_38_round['전북'] < df_38_round['광주']) &
        (df_38_round['전북'] < df_38_round['대전']) &
        (df_38_round['전북'] < df_38_round['대구']) &
        (df_38_round['전북'] < df_38_round['제주'])
    )
]

# 결과 확인
df_2


,제주,광주,대전,대구,전북,인천
84,51,48,45,42,39,40
87,51,48,45,41,39,42
88,51,46,45,41,40,42
93,52,48,43,42,39,40
96,52,48,43,41,39,42
...,...,...,...,...,...,...
699,51,51,43,43,38,40
700,51,49,43,43,39,40
708,48,51,48,43,38,39
717,49,51,46,43,38,39


In [18]:
117 / 729 * 100

16.049382716049383

In [19]:
# 3) '전북' 점수가 가장 낮은 DataFrame
df_3 = df_38_round[
    (
        (df_38_round['전북'] == df_38_round['인천']) &
        (df_38_round['전북'] < df_38_round['광주']) &
        (df_38_round['전북'] < df_38_round['대전']) &
        (df_38_round['전북'] < df_38_round['대구']) &
        (df_38_round['전북'] < df_38_round['제주'])
    )
]

# 결과 확인
df_3


,제주,광주,대전,대구,전북,인천
81,51,48,45,44,39,39
85,51,46,45,42,40,40
90,52,48,43,44,39,39
94,52,46,43,42,40,40
99,54,48,42,44,39,39
...,...,...,...,...,...,...
687,49,51,44,44,38,38
696,51,51,43,44,38,38
709,48,49,48,43,39,39
718,49,49,46,43,39,39


In [20]:
63 / 729 * 100

8.641975308641975

In [23]:
# 인천이 골득실로 12위가 갈릴 경우
df_4 = df_38_round[
    (
        (df_38_round['인천'] == df_38_round['광주']) |
        (df_38_round['인천'] == df_38_round['대전']) |
        (df_38_round['인천'] == df_38_round['대구']) |
        (df_38_round['인천'] == df_38_round['제주'])
    )
]

# 결과 확인
df_4.shape


(54, 6)

In [24]:
# 전북이 골득실로 12위가 갈릴 경우
df_5 = df_38_round[
    (
        (df_38_round['전북'] == df_38_round['광주']) |
        (df_38_round['전북'] == df_38_round['대전']) |
        (df_38_round['전북'] == df_38_round['대구']) |
        (df_38_round['전북'] == df_38_round['제주'])
    )
]

# 결과 확인
df_5.shape


(78, 6)

In [26]:
# 결과 확인
print(df_4.shape)
print(df_5.shape)

print(54 + 78)

(54, 6)
(78, 6)
132


In [27]:
(486 + 117 + 63) / 729

0.9135802469135802

In [31]:
# 전북, 인천을 제외한 팀이 12위가 될 경우의 수
df_6 = df_38_round[
    (
        ((df_38_round['대전'] < df_38_round['인천']) & (df_38_round['대전'] < df_38_round['전북'])) |
        ((df_38_round['제주'] < df_38_round['인천']) & (df_38_round['제주'] < df_38_round['전북'])) |
        ((df_38_round['대구'] < df_38_round['인천']) & (df_38_round['대구'] < df_38_round['전북'])) |
        ((df_38_round['광주'] < df_38_round['인천']) & (df_38_round['광주'] < df_38_round['전북']))
    )
]

# 결과 확인
df_6.shape


(36, 6)

In [32]:
36 / 729

0.04938271604938271

In [101]:
1311 / 19683

0.06660570035055632